# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [0]:
# Reinstalling torch with the right CUDA bindings.
# !pip3 install -U https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
# !pip3 install -U https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [0]:
# Basic imports.
import os
import time
import numpy as np
import torch

import torch
from torch.autograd import Variable
import torch.nn.functional as F

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

## Casting para o dispositivo correto

Como usaremos processamento vetorial principalmente em GPUs para aprendizado profundo, primeiramente é possível verificar se há uma GPU disponível com o trecho de código abaixo, armazenando os tensores nos dispositivos apropriados.

In [0]:
# Checking if GPU/CUDA is available.
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

# O perceptron e a camada linear
A camada Linear do Pytorch ([nn.Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear)) é responsável por aplicar uma transformação linear no dado de entrada. Esta camada recebe como parâmetro a dimensão (número de *features*) da entrada e da saída. Por padrão o bias já é incluído. Um perceptron pode ser facilmente representado como a seguir, desconsiderando a função de ativação:

```
perceptron = nn.Linear(in_dimension, 1)
```
Mas de uma forma geral, uma camada Linear com diversas *features* de entrada e diversas *features* de saída pode ser representada como:
```
nn.Linear(in_features, out_features)
```
![](./figs/nn_linear.png)

In [0]:
linear = nn.Linear(2,3)
print(linear)

Como é possível ver no código abaixo, o Pytorch já inicia os pesos da camada aleatoriamente.

In [0]:
for p in linear.parameters():
  print(p)

O **forward** consiste em passar seu dado de entrada pela rede, gerando um resultado ao final. Considerando a camada linear instanciada anteriormente, o resultado do forward é o mesmo do somatório da multiplicação de seus pesos pelas respectivas entradas juntamente com o bias:

w1\*x1 + w2\*x2 + ... + wn\*xn + b

No Pytorch, realizamos o **forward** chamando a função onde nossa rede/modelo está instanciada, conforme exemplo abaixo.

In [0]:
perceptron = nn.Linear(2,1)
X = torch.FloatTensor([2,3]) #dado de entrada de exemplo considernado o perceptron definido como nn.Linear(2,1)
print('Pytorch: ', perceptron(X))

# acessamos os pesos do modelo com .weight e o bias com .bias
print('Manual: ', torch.mul(X, perceptron.weight).sum() + perceptron.bias)

# Exemplo Perceptron simples
Modifique o código abaixo para que ele realize o forward na camada Linear e calcule a loss

In [0]:
def loss_fn(predict, label):
    return torch.pow(label - predict, 2)

perceptron = nn.Linear(1,1) # Camada linear com 1 feature de entrada (mais o bias) e uma de saída
perceptron.to(device) # casting do perceptron para GPU
learning_rate = 0.01
print('Parametros iniciais: ', list(perceptron.parameters()))

dataset = [] # dados de exemplo quer seguirão a função y = 2x+3
for x in range(10):
    dataset.append((x, 2*x+3))

for epoch in range(101):
    epoch_loss = 0
    for iteration, data in enumerate(dataset):
        X, y = data
        X, y = torch.FloatTensor([X]).to(device), torch.FloatTensor([y]).to(device) # conversão para Tensor
        
        ###########
        # IMPLEMENTE AQUI SUA PARTE DA SOLUÇÃO
        ###########

        epoch_loss += loss.item()
        loss.backward()
        with torch.no_grad():
            for param in perceptron.parameters():
                param -= learning_rate * param.grad # atualização dos parametros (pesos e bias) com base no gradiente
                param.grad.zero_() # resetando o gradiente

    if epoch % 10 == 0:
        print("Epoch {} - loss: {}".format(epoch, epoch_loss))
print('Parametros finais: ', list(perceptron.parameters()))

In [0]:
print(perceptron(torch.FloatTensor([20]).to(device))) #forward do valor 20 para conferir resultado

# Sequential
Na prática, criaremos redes com diversas camadas. O bloco nn.Sequential permite agrupar as camadas de forma sequencial para que o forward seja realizado na ordem desejada automaticamente. Veja um exemplo para um *Multilayer Perceptron (MLP)* abaixo.

In [0]:
in_features = 28
hidden_1 = 64
hidden_2 = 32
out_features = 8

MLP = nn.Sequential(nn.Linear(in_features, hidden_1), nn.ReLU(), 
                    nn.Linear(hidden_1, hidden_2), nn.ReLU(), 
                    nn.Linear(hidden_2, out_features))
print(MLP)

In [0]:
test_data = torch.randn((10,28)) # 10 dados de input aleatórios com 28 features
output = MLP(test_data) # forward da rede
print(output.size())

## Treinando uma MLP simples em dados aleatórios

In [0]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Casting tensors to the appropriate device.
x = x.to(device)
y = y.to(device)

# Printing sizes of tensors.
print('x: ', x.size())
print('y: ', y.size())

## Definindo arquitetura, loss e otimizador
Modifique o código abaixo para criar um container *Sequential* de nome **model** que representa uma MLP com uma camada escondida (seguindo os valores N, D_in, H e D_out definidos anteriormente),**usando um ReLU como função de ativação entre as camadas**.

**Dica:** não se esqueça de realizar o casting da rede para GPU com o comando .to(device)

![](./figs/mlp.png)

In [0]:
# Use the nn package to define our model.
model = # IMPLEMENTE AQUI SUA SOLUÇÃO

print(model)

In [0]:
# Use the nn package to define our loss function.
loss_mse = nn.MSELoss(reduction='sum').to(device)

In [0]:
# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use SGD; the optim package contains many other
# optimization algorithms. The first argument tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## Minimizando o erro entre $f(x)$ e $y$

Implemente abaixo a forward e cálculo da loss.

In [0]:
# Creating list of losses for each epoch.
loss_list = []

# Iterating over epochs.
for epoch in range(500):
    
    ###########
    # Implemente sua solução aqui
    ###########
    
    if (epoch + 1) % 10 == 0:
        print('Epoch ' + str(epoch + 1) + ': loss = ' + str(loss.item()))
    
    # Updating list of losses for printing.
    loss_list.append(loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

In [0]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

ax.plot(np.asarray(loss_list))

plt.show()

Informação sobre outras camadas lineares, como nn.Bilinear e nn.Identity, podem ser vistas na documentação: https://pytorch.org/docs/stable/nn.html#linear-layers